## Med64 QQ Export D3 Data ##
- To use with CoLab notebooks, upload the data files manually. 
- This will downsample the data and export in a file for uploading to observable.
- updated to work on 2022 environmane with 2021 data
- ToDo: fix Pandas df.append deprecated warning

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

import pandas as pd
from functools import partial
from collections.abc import Iterable

In [2]:
from matplotlib import cm
cmap = cm.get_cmap('tab20')

In [3]:
#change pandas.append to pandas.concat

import warnings
warnings.filterwarnings('ignore')

## Warning to fix
```
  df = df.append(unit_df, ignore_index=True)
/var/folders/pd/kpz7k_8d1kvcgy48533dtknw0000gn/T/ipykernel_22571/1779097193.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
```

## Find local data filenames ##

In [4]:
# define global file path

# DATA_DIR = '.' # colab manual import
DATA_DIR = '/Users/walter/Data/margolis/MED64_exampledata_09-22/' # walter local

In [5]:
from med64_data_v0 import *

In [23]:
### KEEP ###
# Keep this for constant used in num_new_points()

max_d3_rows = 10000
MAX_SAMPLES = 32 # default when no df for get_max_samples(df, max_d3_rows)

In [6]:
expt_list, tx_files, unit_files = find_expt_files(DATA_DIR)
datafile_args = dict(data_dir = DATA_DIR, unit_files = unit_files, tx_files = tx_files)

FOUND Data FILES:
Experiment IDs: 
	825_12h24m37s
	825_13h36m25s
	825_15h01m13s
	825_16h06m19s
	827_12h06m26s
	827_13h30m23s
	827_15h57m53s
Units Files: 
	units_825_12h24m37s.mat
	units_827_15h57m53s.mat
	units_825_13h36m25s.mat
	units_827_13h30m23s.mat
	units_827_12h06m26s.mat
	units_825_16h06m19s.mat
	units_825_15h01m13s.mat
Treatment Files: 
	treatmentinfo2020_825_13h36m25s.csv
	treatmentinfo2020_827_13h30m23s.csv
	treatmentinfo2020_827_12h06m26s.csv
	treatmentinfo2020_825_12h24m37s.csv
	treatmentinfo2020_827_15h40m00s.csv
	treatmentinfo2020_827_15h57m53s.csv
	treatmentinfo2020_825_16h06m19s.csv
	treatmentinfo2020_825_15h01m13s.csv


## QQ Plot ##
- get timestamps for tx
- rank timestamps as lin_space from 0 to  1
- rank time in tx as lin_space from start to finish
- plot (ranks of tx, ranks of timestamps )

In [7]:
expt_id = expt_list[0]
units_data, tx_times = get_expt_data(expt_id = expt_id, **datafile_args)

### Firing Rate Calc

In [8]:
def mean_firing_rates(units_data, tx_times):
    """
    get firing rate of a units in Hz
    depricated
    """
    num_units = len(units_data)
    start_time_sec = list(tx_times.values())[0]
    end_time_sec = list(tx_times.values())[-1]
    duration_s = end_time_sec - start_time_sec
    
    mean_firing_rate_hz = dict()
    all_unit_activity_count = 0
    for (id, data) in units_data.items():
        timestamps = data['timestamps']
        activity_count = timestamps.shape[0]
        mean_firing_rate_hz[id] = activity_count / duration_s
        all_unit_activity_count += activity_count
        
    all_unit_mean_fr_hz = all_unit_activity_count / duration_s / num_units
    return all_unit_mean_fr_hz, mean_firing_rate_hz
    
expt_mean_fr, unit_mean_fr = mean_firing_rates(units_data, tx_times)

## Next cell needs following get_tx_ranges() code executed first

In [9]:
def get_mean_firing_by_tx(units_data, tx_times):
    """
    get mean firing rate of units in Hz for each treatment (tx)
    """
    tx_range = get_tx_ranges(tx_times)
    tx_mean_firing_rate_hz = dict()
    for tx in tx_range.keys():
        tx_mean_firing_rate_hz[tx] = {}
    for (unit_id, data) in units_data.items():
        for (tx, [start_time, end_time]) in tx_range.items():
            timestamps = data['timestamps']
            duration = end_time - start_time
            activity_count = [ts for ts in units_data[unit_id]['timestamps'] if 
                 (ts > start_time) and (ts < end_time)]
            tx_mean_firing_rate_hz[tx][unit_id] = len(activity_count)/duration
    return tx_mean_firing_rate_hz

In [10]:
def get_firing_rate_df(df_d3, mean_firing_by_tx):
    """
    create new dataframe with firing rate data formatted to qq dataframe
    """
    firing_rate_df = pd.DataFrame(columns = (df_d3.columns.tolist() + ['Firing Rate']))
    df_index = 0
    for tx in mean_firing_by_tx.keys():
        for unit in mean_firing_by_tx[tx].keys():
            fr = mean_firing_by_tx[tx][unit]
            firing_rate_df.loc[df_index] = ([np.nan, unit, tx] + [np.nan] * 7 + [fr])
            df_index += 1
    return firing_rate_df

In [11]:
#  next line is to create from raw file input
def get_mean_firing_rates(expt_id, datafile_args):
    units_data, tx_times = get_expt_data(expt_id = expt_id, **datafile_args)
    mean_firing_by_tx = get_mean_firing_by_tx(units_data, tx_times)
    return mean_firing_by_tx

### Continue with QQ calc

# A more accurate versoin of get_tx_ranges  
* Use this when I have time to modify the d3 code to display it.
* Also, check if there are timestamps beyond the "TTX off" and allow for arbitrary end time to last range


In [83]:
# THIS IS ACCURATE VERSION DONT DELETE

# import operator

# def get_tx_ranges(tx_times):
#     tx_ranges = {}
#     sorted_tx_times = sorted(tx_times.items(), key=operator.itemgetter(1))
#     prev_tx = None
#     prev_time = None
#     for tx, time in sorted_tx_times:
#         if (prev_tx != None) and (prev_time != time):
#             tx_ranges[prev_tx.strip()] = [prev_time, time]
#         prev_tx = tx
#         prev_time = time
#     return tx_ranges

# get_tx_ranges(tx_times)

{'Baseline': [0.42, 615.0],
 'AMPA blocker DNQX': [615.0, 1215.0],
 'BT139 1uM On': [1215.0, 1635.0],
 'BT139 1uM Off': [1635.0, 2635.0],
 'DAMGO 500nM On': [2635.0, 2935.0],
 'TTX On': [2935.0, 3235.0]}

In [12]:
def get_tx_ranges(tx_times):
    all_times = sorted(list(tx_times.values()))
    last_time_index = len(all_times) - 1
    tx_range = dict()
    for tx, time in tx_times.items():
        time_index = all_times.index(time)
        if time_index == last_time_index:
            break
        next_time = all_times[time_index + 1]
        if time >= next_time:
            continue
        tx_range[tx.strip()] = [time, next_time]
    return tx_range

### Speed up get_timestamp_...
by using lookup in a table generated by array functions instead of loops

In [14]:
def get_timestamp_tx(ts, tx_range):
    for tx, times in tx_range.items():
        if (ts >= times[0]) & (ts < times[1]):
            return tx
    return 'No Tx'
        
def get_timestamp_begin(ts, tx_range):
    for tx, times in tx_range.items():
        if (ts >= times[0]) & (ts < times[1]):
            return times[0]
    return np.nan   
    
def get_timestamp_end(ts, tx_range):
    for tx, times in tx_range.items():
        if (ts >= times[0]) & (ts < times[1]):
            return times[1]
    return np.nan   


In [15]:
def build_df(units_data, tx_times):
    tx_range = get_tx_ranges(tx_times)
    time_tx = partial(get_timestamp_tx, tx_range = tx_range)
    time_begin = partial(get_timestamp_begin, tx_range = tx_range)
    time_end = partial(get_timestamp_end, tx_range = tx_range)

    df = None
    # for loop over units
    for unit_id in list(units_data.keys()):
        timestamps = units_data[unit_id]['timestamps']

        unit_df = pd.DataFrame(timestamps, columns=['timestamp'])
        unit_df['unit'] = unit_id
        if isinstance(df, pd.DataFrame):
            df = df.append(unit_df, ignore_index=True)
        else:
            df = unit_df
    
    # end loop over units
    df['tx'] = df['timestamp'].map(lambda x: time_tx(x))
    df['begin'] = df['timestamp'].map(lambda x: time_begin(x))
    df['end'] = df['timestamp'].map(lambda x: time_end(x))

    ts_groups = df.groupby(by = ['unit', 'tx'])
    df['group_idx'] = ts_groups['timestamp'].cumcount()+1
    
    ts_groups = df.groupby(by = ['unit', 'tx'])
    df['cum_dist'] = ts_groups['group_idx'].apply(lambda df: df/df.count())
    
    return df


In [16]:
def add_anchors(df):
    anchors_begin = df[['unit', 'tx', 'begin', 'end']].drop_duplicates()
    anchors_begin.dropna(inplace=True)
    anchors_end = anchors_begin.copy()
    
    anchors_begin['cum_dist'] = 1
    anchors_begin['timestamp'] = anchors_begin['end']

    anchors_end['cum_dist'] = 0
    anchors_end['timestamp'] = anchors_end['begin']
    
    return pd.concat([df, anchors_begin, anchors_end], axis=0, ignore_index=True)

## Export QQ data for d3 Linked Brushing Cross-filtering    

In [17]:
def data_to_qq(expt_id, datafile_args):
    units_data, tx_times = get_expt_data(expt_id = expt_id, **datafile_args)

    df = build_df(units_data, tx_times)
    df = add_anchors(df)

    df = df[df['tx'] != 'No Tx'] # drop times outside treatment time ranges
    df['x_plot'] = (df['timestamp']-df['begin'])/(df['end']-df['begin'])
    
    # sort zero anchors to beginning of dataframe
    df.sort_values(by=['unit', 'x_plot'], inplace=True)
    
    return df

In [18]:
def get_max_samples(df, total_rows):
    """Returns the max samples per unit per treatment for d3 observable total_rows
    """
    max_d3_rows = total_rows
    max_samples = round(max_d3_rows / df.unit.unique().size / df.tx.unique().size)
    return max_samples

In [19]:
def downsample_data(df, max_d3_rows):
    """downsamples the data in each unit so that the max total size of data will
    work in d3 plot. If more units, then data per unit is larger.
    """
    max_d3_rows = 10000
    max_samples = round(max_d3_rows / df.unit.unique().size / df.tx.unique().size)
    df_downsampled = None

    downsample_groups = df.groupby(by=['unit', 'tx'])

    for by, grp in downsample_groups:
    #     if (by[0] == 5): # & (by[1] == 'DAMGO 500nM On'):
        grp.sort_values(by='x_plot', inplace=True)
        samples = grp.shape[0]
        downsample_rate = round(np.floor(samples / max_samples))
        downsample_rate = 1 if downsample_rate < 1 else downsample_rate
        if isinstance(df_downsampled, pd.DataFrame):
            df_downsampled = df_downsampled.append(grp.iloc[:-1:downsample_rate])
        else:
            df_downsampled = grp.iloc[:-1:downsample_rate] # don't add anchor here
        # add final anchor now, because downsampling could have excluded it
        df_downsampled = df_downsampled.append(grp.iloc[-1:])
#         print(downsample_rate, samples)
#         print(grp.iloc[::downsample_rate])
#         print(grp.iloc[::downsample_rate].shape)
#         print(df_downsampled.shape)
    return df_downsampled

In [20]:
def hypoactive_unit_tx(df, activity_threshold):
    # Depricated for activity_gaps_unit_tx
    """return list of tuples with (unit, tx) for low unit
    activity that will not have enough points on d3 lines"""
    threshold_fraction = 0.5
    group_sizes = df.groupby(by=['unit', 'tx'])['timestamp']
    return [i for i, v in (group_sizes.count() < (threshold_fraction * activity_threshold)).items() if v]

# interp_units = hypoactive_unit_tx(df_downsampled, get_max_samples(df, 10000))

In [21]:
# group_activity = df.groupby(by=['unit', 'tx'])[['timestamp', 'begin', 'end']]
# num_segments = int(get_max_samples(df, 10000)/2)

 # activity_gaps_unit_tx(df_downsampled, get_max_samples(df, 10000)):
    
def groups_with_gaps(group_activity, num_segments): 
    gap_groups = []
    for i, v in group_activity:
#     if i[0] < 3:
#         print(i)
        spikes = v.timestamp.values
        begin = v.iloc[0,:]['begin']
        end = v.iloc[0,:]['end']
        segments = np.linspace(begin, end, num_segments)
        prev_seg = segments[0]
        is_gap = False
        for seg in segments[1:]:
            gap_activity = [s for s in spikes if ((s > prev_seg) & (s < seg))]
#             print(len(gap_activity))
            if len(gap_activity) == 0:
                is_gap = True
        if is_gap:
            gap_groups.append(i)
    return gap_groups
    
# groups_with_gaps(group_activity, num_segments)expt_id

In [22]:
def activity_gaps_unit_tx(df, d3_sample_threshold):
    """return list of tuples with (unit, tx) for unit with gaps in
    activity that will not have enough points on d3 lines"""
    num_segments = int(get_max_samples(df, 10000)/2)
    group_activity = df.groupby(by=['unit', 'tx'])[['timestamp', 'begin', 'end']]
    return groups_with_gaps(group_activity, num_segments)

In [24]:
# df_tmp = df_line[['cum_dist', 'x_plot']].to_numpy()

def euclidist(xy_array):
    """Takes array of x,y coord and calculates the euclidean distance between 
    successive pairs.
    returns array of distances. first is zero, to maintain same length as coord array.
    """
#     min_dist = 1.5 / len(xy_array)
    dist = np.array([0])
    prev_row = xy_array[0]
    for row in xy_array[1:]:
        x_diff = row[0] - prev_row[0]
        y_diff = row[1] - prev_row[1]
        dist = np.append(dist, np.sqrt(x_diff**2 + y_diff**2))
        prev_row = row
#     num_new_points = np.floor(np.array(dist / min_dist))
    return dist

def num_new_points(dist_array, max_d3_samples=MAX_SAMPLES):
    """returns number of points to be added before each index of a
    distance array.
    Assumes total distance of 2 (between sqrt(2) and 2) for qq plot.
    Adjust this number to change density of new points added.
    """
    expected_total_dist = 2 #1.5
    min_dist = expected_total_dist / max_d3_samples
    return np.floor(np.array(dist_array / min_dist))

# interp_input = np.column_stack((df_tmp, num_new_points(euclidist(df_tmp))))
# interp_input

In [25]:
def interp_points(xy_array, num_new_points):
    x0, y0 = xy_array[0]
    new_points = None
    iterator = enumerate(num_new_points)
    for i, n in iterator:
        if (i != 0):
            if (n > 0):
                x0, y0 = xy_array[i - 1]
                x1, y1 = xy_array[i]
                # divide new points along longest of x or axes
                run = x0 - x1
                rise = y0 - y1
                if ((rise/run) < 1):
                    _x = np.linspace(x0, x1, num=int(n + 2))
                    _x = _x[1:-1]  # remove end points because they're not new points
                    _y = np.interp(_x, [x0, x1], [y0, y1])
                else:
                    _y = np.linspace(y0, y1, num=int(n + 2))
                    _y = _y[1:-1]  # remove end points because they're not new points
                    _x = np.interp(_y, [y0, y1], [x0, x1])
                interp_coords = np.column_stack((_x, _y))
                if not isinstance(new_points, np.ndarray):
                    new_points = interp_coords
                else:
                    new_points = np.vstack((new_points, interp_coords))
#     print("interpolating " + str(len(new_points)) + " new points")                    
    return new_points 

# interp_points(df_tmp, num_new_points(euclidist(df_tmp)))

In [26]:
def get_interp_points_df(df_downsampled, interp_units):
    """Takes dataframe of processed points for d3 of qq plots.
    returns another df with same columns but interpolated 
    data points to fill in gaps in raw data. Interpolated points
    aid in selecting in interactive d3 charts.
    """
    tmp_downsampled_df = df_downsampled.iloc[0, :].copy()
    col_names = df_downsampled.columns
    # print(col_names)
    # print(col_names.to_list().index('cum_dist'))

    interp_points_for_df = None

    for unit, tx in interp_units:
#         print("Unit " + str(unit) + " " + tx)
        df_line = df_downsampled[(df_downsampled.unit==unit) & (df_downsampled.tx==tx)]
        xy_array = df_line[['x_plot', 'cum_dist']].to_numpy()
    #     line_euclidist = euclidist(df_line[['cum_dist', 'x_plot']].to_numpy())
        new_points = interp_points(xy_array, num_new_points(euclidist(xy_array)))
        if isinstance(new_points, Iterable):
        #     print("orig num points " + str(len(xy_array)))
        #     print(new_points)
            base_row = df_line.iloc[0:1,:].copy()
            base_row['timestamp'] = np.nan
            collect_rows = base_row  # remove this first dimension holding row after loop
        #     print(base_row.values)
            for x, y in new_points:
                x_index = col_names.to_list().index('x_plot')
                y_index = col_names.to_list().index('cum_dist')
                new_row = base_row.values
                new_row[0][x_index] = x
                new_row[0][y_index] = y 
                collect_rows = np.vstack((collect_rows, new_row))
            collect_rows = collect_rows[1:]
            if not isinstance(interp_points_for_df, np.ndarray):
                interp_points_for_df = collect_rows
            else:
                interp_points_for_df = np.vstack((interp_points_for_df, collect_rows))
        
    print("total interpolated points " + str(interp_points_for_df.shape))      
    interp_df = pd.DataFrame(data = interp_points_for_df, columns = col_names)

    return interp_df

## to do   
- add new dataframe of interp points
- resort dataframe with interp points
- test in d3 observable
- try exporting without padding NaN. it might just work!

### Process final array to export for d3

In [27]:
def clean_column_header(df):
    """Removes column headers not needed for export.
    Changes name of x-value header."""
    df_d3 = df.copy()
    df_d3.drop(["begin", "end", "group_idx"], axis=1, inplace=True)
    df_d3.rename(columns={"x_plot": "x_idx"}, inplace=True)
    return df_d3

In [28]:
# ADD NaN columns. Inefficient. unneeded?
# create row with full features for csv export
# this takes a second
# are all the NaN necessary? I think javascript removes them now...

def pad_data(df):
    """This pads the data with empty columns all filled with NaN values.
    This is the format currently used by the d3 chart."""
    df_d3 = df.copy()
    tx_list = df_d3.tx.unique()

    for tx in tx_list: 
        df_d3[tx] = df_d3[['tx', 'cum_dist']].apply(
            lambda df, tx: df['cum_dist'] if (df['tx']==tx) else np.nan,
            axis=1, tx=tx)
    return df_d3

## Run by steps

In [29]:
df = data_to_qq(expt_id, datafile_args)
df_downsampled = downsample_data(df, max_d3_rows)

In [31]:
# troubleshooting to verify integrity of data at intermediate steps
df[(df.unit==2) & (df.tx=='AMPA blocker DNQX')]
df_downsampled[(df_downsampled.unit==2) & (df_downsampled.tx=='AMPA blocker DNQX')]

,timestamp,unit,tx,begin,end,group_idx,cum_dist,x_plot
273887,615.000000,2,AMPA blocker DNQX,615.0,1215.0,NaN,0.000000,0.000000
29443,620.846636,2,AMPA blocker DNQX,615.0,1215.0,13.0,0.025097,0.009744
29456,624.685607,2,AMPA blocker DNQX,615.0,1215.0,26.0,0.050193,0.016143
29469,626.911028,2,AMPA blocker DNQX,615.0,1215.0,39.0,0.075290,0.019852
29482,627.811075,2,AMPA blocker DNQX,615.0,1215.0,52.0,0.100386,0.021352
29495,630.651542,2,AMPA blocker DNQX,615.0,1215.0,65.0,0.125483,0.026086
29508,637.491729,2,AMPA blocker DNQX,615.0,1215.0,78.0,0.150579,0.037486
29521,648.561355,2,AMPA blocker DNQX,615.0,1215.0,91.0,0.175676,0.055936
29534,659.206869,2,AMPA blocker DNQX,615.0,1215.0,104.0,0.200772,0.073678
29547,673.142523,2,AMPA blocker DNQX,615.0,1215.0,117.0,0.225869,0.096904


In [30]:
# interp_units = hypoactive_unit_tx(df_downsampled, get_max_samples(df, 10000))
max_d3_samples = get_max_samples(df, 10000)
interp_units = activity_gaps_unit_tx(df, max_d3_samples)
interp_points_df = get_interp_points_df(df_downsampled, interp_units)

df_d3 = df_downsampled.append(interp_points_df)

# sort interpolated date into order of full dataframe
df_d3.sort_values(by=['unit', 'x_plot'], inplace=True)

total interpolated points (482, 8)


In [31]:
df_d3 = clean_column_header(df_d3)
df_d3 = pad_data(df_d3)

In [32]:
df.tx.unique()

array(['Baseline', 'AMPA blocker DNQX', 'BT139 1uM On', 'BT139 1uM Off',
       'DAMGO 500nM On'], dtype=object)

In [35]:
df_d3[(df_d3.unit==2) & (df_d3.tx=='AMPA blocker DNQX')] # DAMGO 500nM On

,timestamp,unit,tx,cum_dist,x_idx,AMPA blocker DNQX,BT139 1uM Off,BT139 1uM On,Baseline,DAMGO 500nM On
273887,615.0,2,AMPA blocker DNQX,0.0,0.0,0.000000,NaN,NaN,NaN,NaN
29443,620.846636,2,AMPA blocker DNQX,0.025097,0.009744,0.025097,NaN,NaN,NaN,NaN
29456,624.685607,2,AMPA blocker DNQX,0.050193,0.016143,0.050193,NaN,NaN,NaN,NaN
29469,626.911028,2,AMPA blocker DNQX,0.07529,0.019852,0.075290,NaN,NaN,NaN,NaN
29482,627.811075,2,AMPA blocker DNQX,0.100386,0.021352,0.100386,NaN,NaN,NaN,NaN
29495,630.651542,2,AMPA blocker DNQX,0.125483,0.026086,0.125483,NaN,NaN,NaN,NaN
29508,637.491729,2,AMPA blocker DNQX,0.150579,0.037486,0.150579,NaN,NaN,NaN,NaN
29521,648.561355,2,AMPA blocker DNQX,0.175676,0.055936,0.175676,NaN,NaN,NaN,NaN
29534,659.206869,2,AMPA blocker DNQX,0.200772,0.073678,0.200772,NaN,NaN,NaN,NaN
29547,673.142523,2,AMPA blocker DNQX,0.225869,0.096904,0.225869,NaN,NaN,NaN,NaN


In [33]:
def export_d3_data(df_d3, expt_id):
    save_csv_filepath_dir = '/Users/walter/Data/margolis/observable/'
#     save_csv_filepath = save_csv_filepath_dir + 'qq_for_d3_10K_interp_' + expt_id + '.csv'

    save_csv_filepath = save_csv_filepath_dir + 'd3_cum_fr_10K_interp_' + expt_id + '.csv'

    df_d3.to_csv(save_csv_filepath, index = False)

    print('\tSaved ' + save_csv_filepath)

In [34]:
# for expt_id in expt_id = expt_list
max_d3_rows = 10000

def export_from_filelist(expt_list, datafile_args, interp=True):
    for expt_id in expt_list:
        df = data_to_qq(expt_id, datafile_args)
        df_downsampled = downsample_data(df, max_d3_rows)
        if interp:
            interp_units = activity_gaps_unit_tx(df, get_max_samples(df, 10000))
            ### HERE interp
#             interp_units = hypoactive_unit_tx(df_downsampled, get_max_samples(df, 10000))
            interp_points_df = get_interp_points_df(df_downsampled, interp_units)
            df_d3 = df_downsampled.append(interp_points_df)
            # sort interpolated date into order of full dataframe
            df_d3.sort_values(by=['unit', 'x_plot'], inplace=True)
        df_d3 = clean_column_header(df_d3)
        df_d3 = pad_data(df_d3)

        mean_firing_by_tx = get_mean_firing_rates(expt_id, datafile_args) # get_mean_firing_by_tx(units_data, tx_times)
        firing_rate_df = get_firing_rate_df(df_d3, mean_firing_by_tx)
        df_d3 = firing_rate_df.append(df_d3)
        export_d3_data(df_d3, expt_id)

In [35]:
mean_firing_by_tx = get_mean_firing_by_tx(units_data, tx_times)

In [36]:
firing_rate_df = get_firing_rate_df(df_d3, mean_firing_by_tx)
firing_rate_df.append(df_d3).head()

,timestamp,unit,tx,cum_dist,x_idx,AMPA blocker DNQX,BT139 1uM Off,BT139 1uM On,Baseline,DAMGO 500nM On,Firing Rate
0,NaN,0,Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.502522
1,NaN,1,Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.991344
2,NaN,2,Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.956751
3,NaN,3,Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.401380
4,NaN,4,Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175730


In [37]:
export_from_filelist(expt_list[:], datafile_args)

total interpolated points (482, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_825_12h24m37s.csv
total interpolated points (523, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_825_13h36m25s.csv
total interpolated points (78, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_825_15h01m13s.csv
total interpolated points (75, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_825_16h06m19s.csv
total interpolated points (97, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_827_12h06m26s.csv
total interpolated points (68, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_827_13h30m23s.csv
total interpolated points (155, 8)
	Saved /Users/walter/Data/margolis/observable/d3_cum_fr_10K_interp_827_15h57m53s.csv


In [159]:
expt_list[3:]

['825_16h06m19s', '827_12h06m26s', '827_13h30m23s', '827_15h57m53s']

In [190]:
def file_attachment_list(expt_list):
#     save_csv_filepath_dir = '/Users/walter/Data/margolis/observable/'
    attach_list = "["
    for expt_id in expt_list:
        filename = "FileAttachment(\'d3_cum_fr_10K_interp_" + expt_id + ".csv\')"
        attach_list += filename + ",\n"
    attach_list += "]"
#     return attach_list
    print(attach_list)

In [191]:
file_attachment_list(expt_list)

[FileAttachment('d3_cum_fr_10K_interp_825_12h24m37s.csv'),
FileAttachment('d3_cum_fr_10K_interp_825_13h36m25s.csv'),
FileAttachment('d3_cum_fr_10K_interp_825_15h01m13s.csv'),
FileAttachment('d3_cum_fr_10K_interp_825_16h06m19s.csv'),
FileAttachment('d3_cum_fr_10K_interp_827_12h06m26s.csv'),
FileAttachment('d3_cum_fr_10K_interp_827_13h30m23s.csv'),
FileAttachment('d3_cum_fr_10K_interp_827_15h57m53s.csv'),
]
